In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim

### Part 1: Perform Data Wrangling for Geographical Plotting

**Actions Performed:**
- Cleaning: Back and forward fill to deal with NA values
- Generation: Extract latitudes and longitudes from location names
- Structuring: Convert from wide to long form

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
original_df = pd.read_csv("../csv_files/refugee_movements_by_country.csv")

In [ ]:
df_cleaned = original_df.apply(lambda x: x.replace("-",np.nan))
df_cleaned = df_cleaned.backfill(axis = 1)
df_cleaned = df_cleaned.ffill(axis = 1)

In [ ]:
def get_coordinates(address):
    try:
        geolocator = Nominatim(user_agent="MyApp")
        location = geolocator.geocode(address)
        # print("The latitude of the location is: ", location.latitude)
        # print("The longitude of the location is: ", location.longitude)
        return (location.latitude, location.longitude)
    except:
        print("Failed to obtain coordinates for {}".format(address))
        return None
    
# get_coordinates("Singapore")

In [ ]:
df_cleaned["latitude"], df_cleaned["longitude"] = list(zip(*[get_coordinates(location) for location in df_cleaned["Data Geomaster Name"]]))

In [ ]:
df = pd.melt(df_cleaned, id_vars=['Data Geomaster Name',"latitude","longitude"], value_vars=df_cleaned.columns[2:-2].tolist())

In [ ]:
df.columns = ["Country", "Latitude", "Longitude", "Date", "Refugees from Ukraine"]

In [ ]:
df["datetime"] = pd.to_datetime(df["Date"])

In [ ]:
df = df.astype({'Refugees from Ukraine':'int'})

### Part 2: Geographical Visualisation using Plotly

We use a **choropleth map** to visualise the number of refugees escaping Ukraine to its 7 neighbouring countries.

In [ ]:
import pandas as pd
import chart_studio.plotly as py
import plotly.offline as po
import plotly.graph_objs as pg
import plotly.graph_objects as go
import plotly.express as px
import panel as pn

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# po.init_notebook_mode(connected = True)

In [ ]:
df_others, df_ukraine = df[df["Country"] != "Ukraine"], df[df["Country"] == "Ukraine"]

In [ ]:
# data = dict(type = "choropleth", 
#             locations = list(df_others.Country), 
#             locationmode = "country names", 
#             z = list(df_others["Refugees from Ukraine"]), 
#             )

fig = px.choropleth(df_others, 
                  locations="Country",
                  locationmode = "country names",
                  hover_name="Country",
                  animation_frame = "Date",
                  color = "Refugees from Ukraine",
                  scope="europe",
                  height=800,
                  width=1000,
                  color_continuous_scale=px.colors.sequential.Cividis,
                  range_color=[0,1000000],
                  center = {"lat": 48.3794, "lon": 31.1656}, # Ukraine's coordinates
                  fitbounds = "locations",
                  )

fig.update_layout(title_font_size=30, 
                  # paper_bgcolor = 'rgba(0,0,0,0)',
                  # plot_bgcolor='rgba(0,0,0,0)',
                  legend_bgcolor = "#00FF00",
                  legend_bordercolor="#00FF00",
                  # showlegend=True,
                  geo=dict(bgcolor= 'rgba(0,0,0,0)') # change choropleth background to white
                 )

# fig.show()

In [ ]:
def impact_1():
    return pn.Column(pn.pane.Markdown("## Movement of Ukrainian Refugees into Neighbouring Countries"), 
         pn.pane.Markdown("The war has displaced nearly 12 million Ukrainians from their homeland."),
         pn.pane.Markdown("**Poland** has seen the largest influx of Ukrainian refugees, with the number hovering around 3.6 million refugees. \
                             Russia and Romania are close behind, receiving roughly 1 million Ukrainian refugees."),
         pn.pane.Markdown("**Drag** the interactive slider below to see the movement over time. **Zoom in** on the map to see specific countries."),
         pn.pane.Plotly(fig))

main_3().show()